# Uso básico

Los elementos básicos del paquete son los **intervalos**, i.e. "conjuntos" de números reales (posiblemente incluyendo $\pm \infty$) de la forma 

$$
[a,b] := a \leq x \leq b \subseteq \mathbb{R}
$$

## Creando intervalos

Los intervalos son creados utilizando el macro `@interval`, que toma una o dos expresiones:

In [1]:
using ValidatedNumerics
#Pkg.checkout("ValidatedNumerics")

In [2]:
a = @interval(1)

[1.0, 1.0]

In [3]:
typeof(a) 

ValidatedNumerics.Interval{Float64}

In [4]:
b = @interval(1, 2)

[1.0, 2.0]

Esto retorna objetos del tipo parametrizado `Interval`, el objeto básico del paquete.

El constructor `Interval` puede ser usado directamente, pero no es recomendado en general por los autores del paquete, por la siguiente razón:

In [5]:
 a = Interval(0.1, 0.3)

[0.1, 0.3]

In [6]:
b = @interval(0.1, 0.3)

[0.09999999999999999, 0.30000000000000004]

¿Qué está pasando aquí?

Debido a la forma en que trabaja la aritmética de punto flotante, el intervalo `"a"` creado directamente por el constructor *no contiene el verdadero número real 0.1 ni el 0.3*. El macro `@interval`, sin embargo, usa **redondeo directo** para garantizar que los verdaderos 0.1 y 0.3 estén incluidos en el resultado.

## ¿Por qué es necesario el redondeo?

Consideremos el siguiente código en Julia

In [7]:
x = 0.1

0.1

Esto aparentemente almacena el valor 0.1 en una variable `x` del tipo `Float64`. De hecho, sin embargo, almacena un número *un poco diferente* de 0.1, debido a que 0.1 *no puede ser representado en la aritmética de punto flotante binaria, a ninguna precisión*.

Vemos internamente la representación en `Float64` del número 0.1 (para convencernos de esto):

In [8]:
bits(0.1)

"0011111110111001100110011001100110011001100110011001100110011010"

Los últimos 53 bits de estos 64 bits corresponden a la expansión binaria de 0.1, que es 

```
0.000110011001100110011001100110011001100...
```

Vemos que esta expresión es periódica; de hecho, la expansión binaria de 0.1 tiene una repetición infinita de la secuencia de dígitos `1100`. Por lo tanto es *imposible* representar al decimal 0.1 en binario, a *cualquier* precisión.

El valor que en realidad es almacenado en la variable puede ser determinado convenientemente en Julia usando aritmética de precisión arbitraria (`BigFloat`):

In [9]:
big(0.1) # con 256 bits de precisión

1.000000000000000055511151231257827021181583404541015625000000000000000000000000e-01

Entonces, de hecho, el valor es un poquito más grande que 0.1. Por defecto, estos cálculos se hacen en el modo de "round-to-nearest", redondeo al más cercano (`RoundNearest`); es decir, el número de punto flotante representable más cercano a 0.1 es utilizado.

Supongamos ahora que hemos creado un intervalo como 

In [10]:
II = Interval(0.1)

[0.1, 0.1]

Pareciera como si el intervalo contiene el valor real 0.1, pero debido a la discusión que acabamos de hacer vemos que, de hecho, *esto no es así*. Para que contenga el valor real 0.1, los puntos finales del intervalo deben ser redondeados hacia afuera ("redondeo directo"): el límite inferior es redondeado hacia abajo, y el índice superior hacia arriba.

Este redondeo es manejado por el macro `@interval`, que genera intervalos correctamente redondeados:

In [11]:
a = @interval(0.1)

[0.09999999999999999, 0.1]

El valor verdadero 0.1 está contenido correctamente ahora en los intervalos, por lo tanto cualquier cálculo en estos intevalos contendrá el resultado real calculando con 0.1. Por ejemplo si definimos

In [12]:
f(x) = 2x + 0.2

f (generic function with 1 method)

In [13]:
 f(a)

[0.39999999999999997, 0.4]

El resultado contiene correctamente el verdadero 0.4.

Tras bambalinas, el macro `@interval` reescribe la(s) expresión(es) que se le pasan, reemplazando los literales (0.1, 1, etc.) con llamadas a crear intervalos con redondeo correcto, manejado internamente por la función `make_interval`.

Esto nos permite escribir, por ejemplo

In [14]:
@interval sin(0.1) + cos(0.2)

[1.0798999944880696, 1.07989999448807]

y obtener un resultado que es equivalente a 

In [15]:
sin(@interval(0.1)) + cos(@interval(0.2))

[1.0798999944880696, 1.07989999448807]

Esto también puede ser usado para funciones que defina el usuario:

In [16]:
f(x) = 2x

f (generic function with 1 method)

In [17]:
f(@interval(0.1))

[0.19999999999999998, 0.2]

que es equivalente a 

In [18]:
 @interval f(0.1)

[0.19999999999999998, 0.2]

## Ejemplos 

# $\pi$

Se pueden crear intervalos correctamente redondeados que contengan a $\pi$:

In [19]:
 @interval(pi)

[3.141592653589793, 3.1415926535897936]

e introducirlos en expresiones:

In [20]:
@interval(3*pi/2 + 1)

[5.71238898038469, 5.712388980384691]

In [21]:
@interval(3*π/2 + 1) # la belleza de Julia

[5.71238898038469, 5.712388980384691]

### Formas de crear intervalos 

Los intervalos pueden construirse usando racionales:

In [22]:
 @interval(1//10)

[0.09999999999999999, 0.1]

Los reales son convertidos a racionales:

In [23]:
@interval(0.1)

[0.09999999999999999, 0.1]

Pueden usarse Strings:

In [24]:
@interval("0.1"*"2")

[0.19999999999999998, 0.2]

Pueden usarse Strings en forma de intervalos también:

In [25]:
@interval "[1.2, 3.4]"

[1.2, 3.4000000000000004]

Los intervalos pueden ser creados desde variables:

In [26]:
a = 3.6

3.6

In [27]:
b = @interval(a)

[3.5999999999999996, 3.6]

Los límites superiores e inferiores del intervalon pueden 
ser accesados usando los campos `lo` y `hi`:

In [28]:
b.lo

3.5999999999999996

In [29]:
b.hi

3.6

El diámetro (longitud) de un intervalo es obtenido usando `diam(b)`; para números que no pueden ser representados en binario, el diámetro de los recien creados intervalos pequeños corresponde al epsilon (`eps`) de la máquina, en el modo de redondeo `:narrow`: 

In [30]:
diam(b)

4.440892098500626e-16

## Aritmética

Las operaciones aritmética básicas (`+`,`-`,`*`,`/`,`^`) están definidas para pares de intervalos en la forma estándar: el resultado es el intervalo más pequeño que contiene el resultado de la operación con cada elemento en cada intervalo. Esto eso, para dos intervalos $X$ y $Y$ y una operación $\circ$, definimos la operación sobre los dos intervalos por 

$$
X \circ Y := x \circ y: x \in X \quad \text{y} \quad y \in Y
$$

De nuevo, redondeo directo es usado si se necesita. Por ejemplo:

In [31]:
 a = @interval(0.1, 0.3)

[0.09999999999999999, 0.30000000000000004]

In [32]:
b = @interval(0.3, 0.6)

[0.3, 0.6000000000000001]

In [33]:
a + b

[0.39999999999999997, 0.9000000000000001]

### Pequeña aplicación en la física 

Esta aritmética es muy útil para hacer cálculos científicos, ya que introduce el estudio de errores de manera natural. Hagamos un simple ejemplo de esto.

Supongamos que tenemos un circuito eléctrico muy simple, con una fuente de voltaje $V$ , una corriente $i$ y una resistencia $R$. Alguna medición experimental arrojó el siguiente resultado:

$$
i = 2 \pm 0.1,
$$

$$
R = 7 \pm 0.5,
$$

y queremos, utilizando la ley de Ohm $V = iR$, saber cuanto debe ser el voltaje del circuito.

Utilizando aritmética de intervalos, podemos introducir directamente los errores en nuestro cálculo y obtener un valor para el voltaje que toma en cuenta los mismos, y además que pese el error asociado a cada medición dando un resultado en forma de intervalo, donde sabemos que está garantizado que se encuentre el valor para el voltaje.

In [34]:
i = @interval(1.9,2.1)

[1.9, 2.1]

In [35]:
R = @interval(6.5,7.5)

[6.5, 7.5]

In [36]:
V = i * R

[12.349999999999998, 15.750000000000002]

El cual es el resultado correcto, porque el voltaje debe estar incluido en ese intervalo. Si quieremos expresarlo de una forma parecida a como definimos la corriente y la resistencia hacemos lo siguiente:

In [37]:
mid = (V.lo + V.hi)/2
print("V = ",mid,"±",mid-V.lo)

V = 14.05±1.7000000000000028

## Funciones elementales 

Las funciones elementales principales están definidas, actuando sobre argumentos de intervalos. Actualmente están implementadas las funciones: `exp`, `log`, `sin`, `cos`, `tan`, así como sus inversas; en la última versión (0.2) lanzada el 20 de noviembre de 2015, fueron agregadas algunas funciones hiperbólicas como `sinh`, `cosh`, `tanh` y sus inversas. Ejemplo:

In [38]:
sin(@interval(1))

[0.8414709848078965, 0.8414709848078966]

In [39]:
cos(@interval(10))

[-0.8390715290764525, -0.8390715290764524]

In [40]:
tan(@interval(π))

[-1.2246467991473532e-16, 3.216245299353273e-16]

De nuevo, el resultado debe contener el resultado de aplicar la función a cada número real contenido en el intervalo. Actualmente los creadores del paquete están trabajando para implementar redondeo directo a las funciones elementales.